Bienvenue ! Le but de ce tuto est de vous montrer comment utiliser efficacement Pytorch pous créer des modèles plus performants et plus ambitieux

# Préliminaires

Commencons par coder un réseau tout simple sur MNIST

In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


La première étape c'est evidement de load les données. Dans les TP Automatants cette étape était toujours fait pour vous. Essayons de comprendre ce qui se passe vraiement ! 

In [4]:
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True)

Cette commande va telecharger le dataset MNIST. Pour telecharger d'autres datasets, rendez-vous ici : https://pytorch.org/vision/main/datasets.html. 

Pytorch dispose d'une assez grande collection des datasets pour vous simplifier la vie.

Mais c'est quoi comme object la variable  ```dataset``` ? 

In [7]:
type(dataset)

torchvision.datasets.mnist.MNIST

Mhmmm, ça nous aide pas beaucoup tout ça. Qu'est-ce qu'on peut faire? La première étape c'est d'aller voir le site de pytorch : https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Donc, si je comprends bien, dataset est un iterable. Ca veut dire que je peux itérer dans mon dataset dans une boucle for et avoir mes images et mes labels. Et bah c'est super ça ! 

Par ailleurs, dataset implemente aussi une commande spécifique : len(dataset) me donne sa taille ! 

In [8]:
len(dataset)

60000

In [9]:
# Essayons d'itérer pour récuperer le premier élément du dataset
for entry in dataset:
    print(entry)
    break

(<PIL.Image.Image image mode=L size=28x28 at 0x1B1DF3D6EB0>, 5)


On voit donc qu'entry est un tupple, avec une image type PIL en premier argument et un label en second.

## Exercice 1

Utiliser le dataset pour afficher les premières 10 images et leur labels assosiés

Vous avez peut être remarqué que les images qu'on récupére ne sont pas forcement dans les format qu'on veut. Ici on récupère des images type PIl et pas des tenseurs. Par ailleurs, les images ne sont pas batchées, ce qui n'est pas optimal et ne sont pas normalisés

Pour faire ça on va créer un object transforms qui va gérer tout ça

In [73]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1377,), (0.31,))
])

target_transform =torchvision.transforms.Lambda(lambda y: torch.zeros(
    10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))


# Maintenant on va appliquer les transformations . Attention on applique une transformation sur les features et une autre sur les targets
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms, target_transform=target_transform)


Maintenant on va utiliser le DataLoader pour charger les données

In [74]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [75]:
for batch in dataloader:
    print(batch[0].shape) # images
    print(batch[1].shape) # labels 
    break

torch.Size([64, 1, 28, 28])
torch.Size([64, 10])


Maintenant on peut juste itérer sur notre dataloader et récupérer les batch d'images


## Exercice 2 
Sur un batch de 16, affichez les images avec leur labels


# Définition du modèle

In [82]:
# Maintenant on va créer un réseau de neurones simple pour faire de la classification

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

optimize = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.MSELoss()

epochs = 1 #On va juste train sur une epoches pour le moment
for i , (images, labels) in enumerate(dataloader): # enumerate permet de récupérer l'index de l'itération

    outputs = model(images)

    loss = criterion(outputs, labels)
    optimize.zero_grad()
    loss.backward()
    optimize.step()
    if i % 100 == 0:
        print(loss.item())

0.17612811923027039
0.019344935193657875
0.01965055987238884
0.021056750789284706
0.019860411062836647
0.018823888152837753
0.024677567183971405
0.019886113703250885
0.01946461945772171
0.01990092732012272


Super ça marche ! Cette methode a déjà été faite en TP, j'espère qu'elle est claire maintenant. N'hésitez pas de poser des questions si àa l'est pas

On va compléxifier un peu le modèle maintenant

## Utilisation des classes

Les classes sont couramment utilisées pour créer des modèles en PyTorch. Les classes permettent de définir des modèles de manière modulaire, ce qui les rend plus faciles à comprendre, à personnaliser et à réutiliser. Voici comment vous pouvez utiliser des classes pour créer des modèles en PyTorch :

Commencons par créer une classe : Créer une classe de modèle personnalisée qui hérite de nn.Module 

L'héritage de classe est un concept fondamental de la programmation orientée objet (POO) qui permet à une classe (dans ce cas MonModele) d'hériter des attributs et des méthodes d'une autre classe (dans ce cas nn.Module). Cela signifie que MonModele a accès aux fonctionnalités de nn.Module et peut également ajouter ses propres attributs et méthodes spécifiques.

Dans le contexte de l'apprentissage profond en utilisant PyTorch, cette utilisation de l'héritage de classe est essentielle car nn.Module est une classe de base qui fournit des fonctionnalités cruciales pour la création de modèles, notamment :

 - La gestion automatique des paramètres : Les couches et les modules définis dans une classe dérivée de nn.Module sont automatiquement suivis en termes de paramètres (poids et biais), ce qui permet de gérer plus facilement l'initialisation, l'optimisation et le stockage de ces paramètres.

 - La méthode forward : Vous devez définir la méthode forward dans votre classe dérivée. Cette méthode spécifie comment les données passent à travers votre modèle. Elle est utilisée pour effectuer la propagation avant (forward pass) lors de l'inférence.

 - La gestion des sous-modules : Vous pouvez incorporer d'autres modules PyTorch dans votre modèle, les empiler et les combiner en utilisant des opérations mathématiques, tout en bénéficiant de la gestion des paramètres et de la sérialisation automatique fournie par nn.Module.

L'héritage de nn.Module facilite donc la création de modèles personnalisés tout en garantissant la cohérence avec le framework PyTorch. En plus d'hériter des fonctionnalités de base de nn.Module, vous pouvez ajouter des couches spécifiques et des opérations pour créer votre modèle sur mesure, comme cela est illustré dans l'exemple.

In [ ]:
class MonModele(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MonModele, self).__init__()

        # Définir les couches du modèle
        self.couche_entree = nn.Linear(input_size, hidden_size)
        self.couche_cachee = nn.Linear(hidden_size, hidden_size)
        self.couche_sortie = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Définir la passe avant (forward pass) du modèle
        x = self.couche_entree(x)
        x = self.relu(x)
        x = self.couche_cachee(x)
        x = self.relu(x)
        x = self.couche_sortie(x)
        return x

 - Dans le constructeur __init__, vous définissez les couches du modèle en utilisant les modules PyTorch tels que nn.Linear, nn.Conv2d, etc. Vous pouvez également définir d'autres paramètres du modèle dans ce constructeur.

 - Dans la méthode forward, vous spécifiez comment les données passent à travers le modèle. Vous décrivez l'enchaînement des couches et des opérations pour effectuer une propagation avant.

Créez une instance de votre modèle en fournissant les dimensions d'entrée, de sortie et d'autres paramètres requis :

In [ ]:
input_size = 64
hidden_size = 128
output_size = 10
mon_modele = MonModele(input_size, hidden_size, output_size)

Pour effectuer une prédiction avec votre modèle, appelez la méthode forward :

In [ ]:
input_data = torch.randn(32, input_size)  # Exemple de données d'entrée
output = mon_modele(input_data)


Pour entraîner le modèle, vous devez définir une fonction de perte (loss function) et un optimiseur, puis effectuer une rétropropagation pour ajuster les poids du modèle. Exactement comme on a fait avant ! 

Vous pouvez ensuite sauvegarder et charger le modèle en utilisant les méthodes de sérialisation de PyTorch, par exemple, torch.save() et torch.load().

L'utilisation de classes pour définir des modèles en PyTorch rend le code plus organisé, modulaire et plus facile à gérer. Vous pouvez également hériter de modèles pré-entraînés et personnaliser leurs architectures pour des tâches spécifiques en utilisant cette approche.

## Exercice 3 
A vous de jouer ! Coder un modèle simple qui s'entraine à classifier les chiffres sur MNIST